# 이상탐지 알고리즘 : Isolation Forest



## 0.환경준비

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest # Isolation Forest!
from sklearn.metrics import *

from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore')

## 1.Isolation Forest 개념이해

> sklearn help : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html  
Isolation Forest Original Paper : https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest

### (1) 샘플데이터 가져오기

In [ ]:
# Single Blob
X1 = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/Anomaly_X.csv')

# Double Blob
X2 = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/Anomaly_X2.csv')

### (2) 모델 시각화 함수

* 모델 시각화 함수 만들기
    * 실전에서는 사용하기 어렵습니다.
    * 교육적인 이해를 돕고자 사용합니다.(소스코드 문법 몰라도 됩니다!!)

In [ ]:
def model_visualize(model, v1, v2, title = "") :
    # 메쉬그리드값 저장하기
    xx, yy = np.meshgrid(np.linspace(-5, 5, 50), np.linspace(-5, 5, 50)) # mesh grid

    # 메쉬 그리드값에 대해 모델 부터 Anomaly Score 만들기.
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()]) # Anomaly Score
    Z = Z.reshape(xx.shape)
    # 시각화
    plt.figure(figsize = (8,8))
    plt.title(title)

    # 메쉬그리드 값의 Anomaly Score에 대한 등고선
    plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)

    # 데이터 산점도 그리기.(예측 결과 Abnormal은 오렌지색, Normal은 흰색)
    sns.scatterplot(x=v1, y=v2, sizes = 30, edgecolor='k', hue = pred, palette=['white', 'orange'])

    plt.axis("tight")
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.show()

### (3) Single Blob

In [ ]:
# sample data
plt.figure(figsize = (8,8))

plt.scatter(X1['v1'], X1['v2'], c="white", s=30, edgecolor="k")

plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid()
plt.show()

* Isolation Forest 모델을 일단 만들어 봅시다.

In [ ]:
# 모델링
model = IsolationForest(contamination = 0.1, n_estimators = 50 )
model.fit(X1)
pred = model.predict(X1)
pred

In [ ]:
pred = np.where(pred == 1, 0, 1)

* 모델을 시각화 해 봅시다.

In [ ]:
model_visualize(model, X1['v1'], X1['v2'], 'Isolation Forest')

* 몇가지를 조절해 봅시다.
    * contamination
        * Score 계산 후, 이상치로 간주할 비율
    * n_estimators
        * 하나의 데이터를 고립시키기 위해 생성하는 tree의 개수
        * Default : 100

In [ ]:
# 여기를 조정해 봅시다.
cont = 0.4
n_est = 100

# 모델링
model = IsolationForest(contamination = cont, n_estimators = n_est
                        , random_state = 20)
model.fit(X1)
pred = model.predict(X1)
pred = np.where(pred == 1, 0, 1)

model_visualize(model, X1['v1'], X1['v2'], 'Isolation Forest')

### (4) Double Blob

In [ ]:
plt.figure(figsize = (8,8))

plt.scatter(X2['v1'], X2['v2'], c="white", s=30, edgecolor="k")

plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid()
plt.show()

* Isolation Forest 모델을 일단 만들어 봅시다.

In [ ]:
# 모델링
model = IsolationForest(contamination = 0.1, n_estimators = 50 )
model.fit(X2)
pred = model.predict(X2)
pred = np.where(pred == 1, 0, 1)

* 모델을 시각화 해 봅시다.

In [ ]:
model_visualize(model, X2['v1'], X2['v2'], 'Isolation Forest')

* 몇가지를 조절해 봅시다.
    * contamination
        * 데이터에서 존재하는 Abnormal 데이터의 비율.
        * Score 계산 후, 이상치로 간주할 비율
    * n_estimators
        * 하나의 데이터를 고립시키기 위해 생성하는 tree의 개수
        * Default : 100

In [ ]:
# 여기를 조정해 봅시다.
cont = 0.03
n_est = 300

# 모델링
model = IsolationForest(contamination = cont, n_estimators = n_est, random_state = 20)
model.fit(X2)
pred = model.predict(X2)
pred = np.where(pred == 1, 0, 1)

# 모델 시각화
model_visualize(model, X2['v1'], X2['v2'], 'Isolation Forest')

## 2.반도체 제조공정간 불량 탐지


![](https://assets.pandaily.com/uploads/2021/10/semiconductor.png)

* 반도체 제조 공정은 시점별로 수많은 센서로부터 정보를 수집하여 공정을 감시합니다.
* 센서정보와 함께 공정간 발생된 불량품에 대한 정보를 저장하였습니다.
* 불량을 예측해 봅시다.

### (1) 데이터 준비

* 데이터 로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/secom_9.csv"
data = pd.read_csv(path)

data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)
data.head()

변수 정보
* label : 1 - 불량, 0 - 정상
* v### : 센서값들


In [ ]:
target = 'label'

In [ ]:
data[target].value_counts() / data.shape[0]

* x, y로 나누기

In [ ]:
x = data.drop(target, axis = 1)
y = data.loc[:, target]

* 데이터 분할
    * 이미 test set은 분할되어 있다고 가정합니다.
    * 주어진 데이터를 train set : validation set 으로 분할

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train_val에서 train : val = 8 : 2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state = 20)

In [ ]:
print(x_train.shape, x_val.shape)

### (2) Step1 : Contamination 조정
* contamination 초기값 : train셋 y의 비율로 지정
* validation 셋으로 예측을 하고, 평가해 봅시다.

#### 1)모델링

In [ ]:
y_train.value_counts(normalize = True)

In [ ]:
# contamination=0.068로 지정하고 모델을 생성해 봅시다.
model = IsolationForest(contamination=0.07, random_state = 20)

# 비지도 학습입니다. x_train만 사용!
model.fit(x_train)

#### 2)예측 및 평가

In [ ]:
# 예측
pred = model.predict(x_val)

# 결과를 1,0으로 변환(np.where)
pred = np.where(pred == 1, 0, 1)

# 분류 결과를 평가해 봅시다.
print(confusion_matrix(y_val, pred))
print(classification_report(y_val, pred))

### (3) 실습
* contamination 값을 조절하면서 최적의 성능(f1)을 갖는 모델을 생성해 봅시다.

## 3.[추가] Scoring 및 최적의 cut-off

### (1) Scoring

* score 점수를 이용하여, 수동으로 cut-off 조절해가며 지표 계산 및 그래프를 그려 봅시다.

In [ ]:
# 학습 데이터에 대한 abnormal 점수 계산
tr_score = model.score_samples(x_train)

# 0~1 의 값으로 score 바꾸기
tr_score = -1 * tr_score

# score의 분포 살펴보기
sns.histplot(tr_score, kde= True)
plt.show()

### (2) 최적의 cuf-off value 찾기

* score 점수를 이용하여, 수동으로 cut-off 조절해가며 지표 계산 및 그래프를 그려 봅시다.
    * prec_rec_f1_curve 함수 이용
    * validation set 이용

In [ ]:
from sklearn.metrics import precision_recall_curve

def prec_rec_f1_curve(y, score, pos = 1) :

    # precision, recall, f1 계산
    precision, recall, thresholds  = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)

    # 그래프 그리기
    plt.figure(figsize = (12,8))
    plt.plot(thresholds, np.delete(precision, -1), label = 'precision')
    plt.plot(thresholds, np.delete(recall, -1), label = 'recall')
    plt.plot(thresholds, np.delete(f1, -1), label = 'f1')

    #-------------------------------------------------------------------------
    # 아래 코드는 차트 꾸미기 입니다.
    # f1를 최대화 해주는 threshold
    thre = round(thresholds[np.argmax(f1)],4)
    f1s = round(max(f1),4)
    plt.axvline(thre , color = 'darkred', linewidth = .7)
    plt.axhline( f1s, color = 'darkred', linewidth = .7)
    plt.text(thre, .5, thre, color = 'darkred')
    plt.text(min(thresholds), f1s, f1s, color = 'darkred')

    plt.xlabel('Anomaly Score')
    plt.ylabel('Performance')
    plt.legend()
    plt.grid()
    plt.show()

    return precision, recall, f1, thresholds

In [ ]:
# validation score 계산
va_score = model.score_samples(x_val)

# 0~1 의 값으로 score 바꾸기
va_score = -1 * va_score

# prec_rec_f1_curve 이용하여 최적의 cutoff 찾기

_, _, f1, thresholds = prec_rec_f1_curve(y_val, va_score)

* f1값을 최대화 하는 cut-off 값을 구해봅시다.

In [ ]:
cutoff_f1max = thresholds[np.argmax(f1)]

In [ ]:
pred = np.where(va_score >= cutoff_f1max,1,0)

In [ ]:
print(confusion_matrix(y_val, pred))
print(classification_report(y_val, pred))

### (3) 하이퍼파라미터 튜닝
* 다음의 하이퍼파라미터를 각각 조절하며, f1 스코어를 구하고, 경향과, 적절한 지점을 파악해 봅시다.
    * n_estimators
    * max_samples

#### 1) contamination 값 찾기

In [ ]:
new_cont = sum(va_score >= cutoff_f1max) / len(va_score)
new_cont

#### 2) n_estimators

In [ ]:
n_est = range(50,501,50)
f1_list = []
for t in tqdm(n_est) :
    f1_temp = []
    for i in range(5) :
        model = IsolationForest(contamination = new_cont, n_estimators = t)
        model.fit(x_train)
        pred = model.predict(x_val)
        pred = np.where(pred == 1, 0, 1)
        f1_temp.append(f1_score(y_val, pred, pos_label=1))

    f1_list.append(np.mean(f1_temp))

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(n_est, f1_list, marker = '.')
plt.ylabel('F1 Score')
plt.xlabel('n_estimators')
plt.grid()
plt.show()

#### 3) max_samples

In [ ]:
m_sam = range(500,5001,500)
f1_list = []
for s in tqdm(m_sam) :
    f1_temp = []
    for i in range(5) :
        model = IsolationForest(contamination = new_cont, n_estimators = 300 , max_samples = s)
        model.fit(x_train)
        pred = model.predict(x_val)
        pred = np.where(pred == 1, 0, 1)
        f1_temp.append(f1_score(y_val, pred, pos_label=1))

    f1_list.append(np.mean(f1_temp))


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(m_sam, f1_list, marker = '.')
plt.ylabel('F1 Score')
plt.xlabel('max_samples')
plt.grid()
plt.show()